In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd

In [2]:
def extract_injury_table(url):
    browser = Browser('chrome')
    browser.visit(url)
    html = browser.html
    html_soup = soup(html, "html.parser")
    
    browser.links.find_by_partial_text("Single Grid").click()
    dropdown = browser.find_by_tag('select')
    dropdown.select_by_text('Infinity')
    
    # Parse the HTML page
    html = browser.html
    html_soup = soup(html, "html.parser")
    tables = html_soup.find_all('div', class_="table-scroll")
    
    # Extract headers and data
    rows = tables[0].find_all('tr')
    headers = [x.text for x in rows[0].find_all('th')]
    
    # Loop through  each table
    injury_table = []
    for table in tables:
        rows = table.find_all('tr')
        for row in rows[1:]:
            injury_table.append([val.text for val in row.find_all('td')])
    
    browser.quit()
    
    # Convert to DataFrame
    df = pd.DataFrame(injury_table, columns=headers)
    
    return df
    
    

In [3]:
# Define lists of URLs
urls = ["https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2024",
        "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2023",
        "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2022",
        "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2021",
        "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2020"]

In [4]:
# Extract injury tables from each URL 
dfs = []
for url in urls:
    year = url.split("=")[-1]  # Extract the year from the URL
    df = extract_injury_table(url)
    df['year'] = year  # Add a new column "year" with the extracted year
    print(f"Table extracted from {url} for year {year}:")
    print(df)
    print("Shape of the DataFrame:")
    print(df.shape)
    print("\n")

    # Append the extracted DataFrame to the list
    dfs.append(df)

# Combine all DataFrames into a single DataFrame
df_combined = pd.concat(dfs)

# Display the combined DataFrame with the year column
print("Combined DataFrame for all years:")
print(df_combined)
print("Shape of the combined DataFrame:")
print(df_combined.shape) 
    
# Define dictionary of URLs and corresponding element IDs to close
urls_elements = {
    "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2024": "element_id_2024",
    "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2023": "element_id_2023",
    "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2022": "element_id_2022",
    "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2021": "element_id_2021",
    "https://www.fangraphs.com/roster-resource/injury-report?groupby=all&timeframe=all&season=2020": "element_id_2020"
}

# Function to close pop-up element if present on a given URL
def close_element_if_present(url, element_id):
    browser = Browser("chrome")
    browser.visit(url)
    
    if browser.is_element_present_by_id(element_id):
        time.sleep(2)                # Wait time before clicking the element
        browser.find_by_id(element_id).first.click()
    
    browser.quit()

    

# Close elements for each URL
for url, element_id in urls_elements.items():
    close_element_if_present(url, element_id)

# Save the DataFrame to a CSV file  - df is saved below


## i am not sure but this bit of code is not running correctly and wont see the chrome driver even know i have tried alot of different methods to make it see it

DriverNotFoundError: Driver for chrome was not found.

In [54]:
df_combined

NameError: name 'df_combined' is not defined

In [7]:
# Find the missing values for each column in the Data
missing_values = df_combined.isnull().sum()
missing_values

NameError: name 'df_combined' is not defined

In [8]:
# Show most common injuries
most_common_injuries = df_combined['Injury / Surgery'].value_counts().idxmax()
most_common_injuries

NameError: name 'df_combined' is not defined

In [9]:
# Show which team has the most injuries and what type of injuries are they
df_team_injuries = df_combined.groupby('Team')['Injury / Surgery'].value_counts()
df_team_injuries


NameError: name 'df_combined' is not defined

In [38]:
# df_combined
# Most common injury for each year
most_common_injuries_year = df_combined.groupby('year')['Injury / Surgery'].value_counts().unstack().idxmax()
most_common_injuries_year

Injury / Surgery
                                          2021
 Vascular surgery (upper arm aneurysm)    2024
AC joint inflammation                     2022
Abdomen contusion                         2023
Abdomen infection                         2024
                                          ... 
Wrist surgery                             2021
Wrist surgery (fracture)                  2022
Wrist surgery (ulnar styloid fracture)    2024
Wrist tendinitis                          2021
Wrist tendonitis                          2023
Length: 681, dtype: object

In [42]:
# df_combined
# Which injury takes a long time to recover from
longest_recovery_time = df_combined.groupby('Injury / Surgery')['Status'].max().sort_values(ascending=False)
longest_recovery_time

Injury / Surgery
Elbow inflammation                 Released
Blister on right hand              Released
Strained hamstring                 Released
Strained calf                   Player Pool
Undisclosed                     Player Pool
                                   ...     
Fractured kneecap                 10-Day IL
Hand surgery (torn ligament)      10-Day IL
Chest contusion                            
Thumb surgery (laceration)                 
                                           
Name: Status, Length: 681, dtype: object

In [43]:
# Save the Dataframe to a CSV file
df_combined.to_csv('injury_report_combined_years_v4.csv', index=False)